In [1]:
# import packages
import os
import glob
import numpy as np
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import auth

### Add user input
Fill in the variables in the cell below with the following information

- `experiment` = name of experiment (outermost collection) in Firebase
- `local_folder` = name of folder you want created locally and populated with data
- `credential_path` = path to .json Firebase SDK service account key (explained [here](https://firebase.google.com/docs/admin/setup))
- `firebase_project_name` = name of project in Firebase (click Project Overview in the Firebase console to see name)
- `groups` = array of group names (as set by userGroup in src/utils.js and found as collections in the 'subject' document in your Firebase), each enclosed in single quotes and separated by commas
- `rating_types` = array of rating types, each enclosed in single quotes and separated by commas (same as ratingTypes from src/utils.js)
- `movie_names` = array of movie names, as written in Firebase stimuli table (no spaces), each enclosed in single quotes and separated by commas
- `vid_lens` = dictionary of video durations, where each entry contains the movie name as key and the movie duration (in seconds) as value (e.g. {'movie1': 100, 'movie2': 150, 'movie3': 120}) 


In [3]:
# USER VARIABLES (FILL IN HERE)
experiment = 'wasabi-online'
local_folder = '/Users/jasondavis/Documents/MATLAB/state_affect_mapping_jad/wasabi_online/'
credential_path = '/Users/jasondavis/Documents/Downloads/continuous-rater-jad-firebase-adminsdk-2ie9g-0080d92b8a.json'
firebase_project_name = 'continuous-rater-jad'
groups = ['Prolific Group'] # possible to only have on group, or can use multiple for organizational purposes
rating_types = ['pleasant', 'unpleasant', 'calm', 'aroused', 'funny', 'happy', 'angry', 'sad', 'disgusted', 'afraid', 'suprised']
movie_names = ['KungFuryPart1', 'KungFuryPart2']
vid_lens = {'KungFuryPart1': 931, 'KungFuryPart2': 931} # example values, fill in with your own

### Set up communication with Firebase

In [4]:
cred = credentials.Certificate(credential_path)
firebase_admin.initialize_app(cred, {
  'projectId': firebase_project_name,
})

db = firestore.client()

### Initialize data structures and directories

In [6]:
# initialize data structures
subject_rating_dict = {}
movie_q_count_dict = {}
sub_to_mov = {}
mov_to_sub = {}

# initialize directory structure
directories = []
directories.append(f'{local_folder}/')
directories.append(f'{local_folder}/Long/')
directories.append(f'{local_folder}/Blanks/')
directories.append(f'{local_folder}/Subjects/')
directories.append(f'{local_folder}/Subjects/Incomplete/')
directories.append(f'{local_folder}/Subjects/Groups/')
directories.append(f'{local_folder}/Summary/')
directories.append(f'{local_folder}/Ratings/')
for movie in movie_names:
    for rating in rating_types:
        combo = movie + "-" + rating
        movie_q_count_dict[combo] = 0
        mov_to_sub[combo] = []
        directories.append(f'{local_folder}/Ratings/{combo}/')
        
for directory in directories:
    if not os.path.isdir(directory):
        os.mkdir(directory) 

### Create blank pandas dataframes of proper shape to fill in with rating data

In [7]:
# use provided movie durations to create blank dataframe of proper length for each movie
blanks_path = f'{local_folder}/Blanks/'
for movie in movie_names:
    curr_vid_len = vid_lens[movie] + 3 # add buffer time
    init_ratings = np.full(curr_vid_len, -1)
    curr_df = pd.DataFrame({'rating': init_ratings})
    curr_df.to_csv(os.path.join(blanks_path, f'{movie}.csv')) # write out to file

In [8]:
# read blank dataframes back in and store in dictionary
blanks_path = f'{local_folder}/Blanks/'
blank_pd_dict = {}
directory_list = glob.glob(os.path.join(blanks_path, '*.csv'))
for file in directory_list:
    curr_df = pd.read_csv(file)
    drop_df = curr_df.drop(labels='Unnamed: 0', axis=1)
    movie = os.path.basename(file).split('.')[0]
    blank_pd_dict[movie] = drop_df

### Determine which subjects completed task, and save subject info to .csv files

In [9]:
# create good and bad subject lists (based on completion)
good_id_master_list = []
bad_id_master_list = []

for group in groups:
    good_id_list = [] # stores participants who completed HIT
    bad_id_list = [] # stores participants who started but didn't complete HIT
    sub_list = []

    group_path = f'{experiment}/subjects/{group}'
    group_collection = db.collection(group_path)
    group_subs = group_collection.stream()

    for sub in group_subs:
        sub_dict = sub.to_dict()
        if 'currentState' in sub_dict: # check to see if subject even started HIT
            if sub_dict['currentState'] == 'debrief' or 'HIT_complete' in sub_dict: # only keep subs who finished
                good_id_list.append(sub.id)
                good_id_master_list.append(sub.id)
                curr = pd.Series(sub_dict)
                sub_list.append(curr)
                file_path = f'{local_folder}/Subjects/{sub.id}.csv'
                curr.to_csv(file_path)
        else:
            bad_id_list.append(sub.id)
            bad_id_master_list.append(sub.id)
            curr = pd.Series(sub_dict)
            file_path = f'{local_folder}/Subjects/Incomplete/{sub.id}.csv'
            curr.to_csv(file_path)

    group_df = pd.DataFrame(sub_list)
    group_df.to_csv(f'{local_folder}/Subjects/Groups/{group}.csv')

I0000 00:00:1736348271.543045 2324159 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1736348271.569067 2324159 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


### Get all ratings from subjects who completed task and store locally

In [10]:
# loops over subjects and gets ratings from database to store in local dictionary
# also counts number of subjects that have rated each question
# links subjects to movies and movies to subjects
good_id_set = set(good_id_master_list) # removes repeats

for good_id in good_id_set: 
    sub_to_mov[good_id] = []
    movie_list = []
    collection_path = f'{experiment}/ratings/{good_id}'
    curr_sub_ratings = db.collection(collection_path)
    HITs = curr_sub_ratings.stream()

    curr_movie_dict = {}
    for HIT in HITs:
        sub_to_mov[good_id].append(HIT.id)
        mov_to_sub[HIT.id].append(good_id)
        curr_movie_dict[HIT.id] = HIT.to_dict()
        movie_q_count_dict[HIT.id] += 1

    if good_id in subject_rating_dict:
        updated = subject_rating_dict[good_id].append(curr_movie_dict)
        subject_rating_dict[good_id] = updated
    else:
        movie_list.append(curr_movie_dict)
        subject_rating_dict[good_id] = movie_list

movie_counts = pd.Series(movie_q_count_dict)
movie_counts.to_csv(f'{local_folder}/Summary/Movie_Counts.csv')

### Write out rating dictionary to .csv rating files

In [11]:
# writes file containing rating-timestamp pairs for each subject to folder for each movie-rating pairing
for good_id in good_id_set:
    curr_sub = subject_rating_dict[good_id]
    for dictionary in curr_sub:
        for movie_rating in dictionary:
            words = movie_rating.split('-')
            blank_mov_pd = blank_pd_dict[words[0]].copy()
            rating_dict = dictionary[movie_rating]
            count = 0
            print(len(rating_dict))
            for timestamp in rating_dict:
                    if timestamp.isdigit():
                        blank_mov_pd.iloc[int(timestamp)] = rating_dict[timestamp]
                    else:
                        continue
            file_path = f'{local_folder}/Ratings/{movie_rating}/{good_id}.csv'  
            blank_mov_pd.to_csv(file_path)      

253
159
2
1
355
545
2
1
464
575
2
7
2
1
89
83
248
206
2
1
2
1
19
20
325
321
2
1
2
1
89
21
61
29
12
9
14
18
14
4
52
22
141
188
38
3
47
5
777
9
56
28
41
19
156
184
206
97
155
105
206
140
10
4
73
9
164
11
197
168
2
1
43
32
415
120
149
72
2
1
2
7
25
41
2
1
299
252
15
7
2
1
190
26
869
868
2
1
97
85
270
307
2
1
155
63
146
121
2
1
265
193
2
1
63
158
22
37
2
1
130
239
2
1
135
99
28
39
146
52
2
1
2
1
9
5
2
173
25
130
182
149
531
331
3
3
2
1
2
1
19
7
41
15
11
9
43
31
75
71
143
71
441
695
496
503
2
1
27
10
169
105
2
10
8
6
2
1
2
1
2
1
40
40
2
1
2
1
2
1
187
431
96
41
28
24
113
109
2
1
28
21
2
1
27
12
2
1
5
3
31
17
2
1
273
242
11
3
2
1
2
1
618
638
2
1
243
175
25
87
15
1
2
1
2
1
17
11
6
4
2
15
50
126
2
1
25
9
440
434
28
25
2
1
2
1
215
200
2
1
4
3
7
1
2
1
2
1
407
531
2
34
60
85
2
1
164
102
2
1
11
4
295
357
36
3
102
20
36
24
298
242
25
12
2
1
80
87
13
5
160
153
53
58
356
336
113
123
2
26
61
57
2
1
2
1
2
75
2
1
16
11
2
53
11
3
11
3
3
1
108
89
147
59
2
1
2
1
2
1
13
25
77
60
127
82
2
1
2
1
2
74


### Combine subject and rating info together to create long format file

In [12]:
# make the base frame for appending
cols = ['workerId', 'movie', 'ratingType', 'HIT_complete', 'age', 'assignmentId', 'birth', 'consentStatus', 'currentState', \
'ethnicity', 'feedback', 'handed', 'hitId', 'nativeLang', 'race', 'sex', 'startTime', \
'userId', 'mostRecentTime', 'timeStamp', 'ratingScore']

base_frame = pd.DataFrame(np.nan, index=[0], columns=cols)

In [14]:
# this take individual local csvs and turns them into long format per movie per rating per subject
dir_list = glob.glob(f'{local_folder}/Ratings/*')
master_long = base_frame.copy()

for directory in dir_list:
    path = directory + '/*.csv'
    rating_list = glob.glob(path)
    for file in rating_list:
        sub_id = os.path.basename(file).split('.')[0]
        movie_rating = file.split('/')[2]
        movie = movie_rating.split('-')[0]
        rating = movie_rating.split('-')[0]       

        # this should check and not rewrite files that already exist (speeds up process)
        if not os.path.isfile(f'{local_folder}/Long/{movie}-{rating}-{sub_id}.csv'):
            subject_long = base_frame.copy()
            rating_pd = pd.read_csv(file)
            sub_path = f'{local_folder}/Subjects/{sub_id}.csv'
            if os.path.isfile(sub_path):
                sub_pd = pd.read_csv(sub_path)
                title_list = sub_pd['Unnamed: 0'].values
                rename_dict = {}
                counter = 0
                for title in title_list:
                    rename_dict[counter] = title
                    counter += 1   
                new_sub_pd = sub_pd.transpose().rename(columns=rename_dict).drop(['Unnamed: 0'])

                new_pd = base_frame.copy()
                new_pd['movie'] = movie
                new_pd['ratingType'] = rating

                for category in base_frame:
                    if category in new_sub_pd:
                        new_pd[category] = new_sub_pd[category].values

                timestamp_dict = rating_pd.transpose().drop(['Unnamed: 0'])
                copy_pd = new_pd.copy()
                prevScore = -1
                rating_counter = 0
                for timestamp in timestamp_dict:
                    ratingScore = timestamp_dict[timestamp].values
                    if ratingScore != -1:
                        prevScore = ratingScore
                    else:
                        ratingScore = prevScore

                    copy_pd['timeStamp'] = timestamp
                    copy_pd['ratingScore'] = ratingScore
                    subject_long = pd.concat([subject_long, copy_pd], ignore_index=True)

                subject_long = subject_long.drop([0])
                subject_long.to_csv(f'{local_folder}/Long/{movie}-{rating}-{sub_id}.csv')                   

In [15]:
# this appends all individual long format files into one giant long format panda
master_long = base_frame.copy()
long_list = glob.glob(f'{local_folder}/Long/*.csv')

for file in long_list:
    curr_pd = pd.read_csv(file)
    master_long = pd.concat([master_long, curr_pd])

master_long.drop([0])
master_long.to_csv(f'{local_folder}/master_long.csv')

In [16]:
# read in saved copy
master_long = pd.read_csv(f'{local_folder}/master_long.csv')
master_long.drop(labels=[0], inplace=True)
master_long.drop(labels=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
sub_list = master_long['workerId'].unique()
movie_list = master_long['movie'].unique()
rating_list = master_long['ratingType'].unique()
movie = movie_list[0]
rating = rating_list[0]

In [31]:
master_long

,workerId,movie,ratingType,HIT_complete,age,assignmentId,birth,consentStatus,currentState,ethnicity,...,handed,hitId,nativeLang,race,sex,startTime,userId,mostRecentTime,timeStamp,ratingScore
1,453829892.0,jasondavis,jasondavis,2024-11-24 23:06:47.564000+00:00,30.0,453829892.0,"Manila, Metro Manila, Philippines",signed,complete,not_hispanic,...,right,453829892.0,Tagalog (Filipino),['Asian / Asian-American'],female,2024-11-24 23:06:47.564000+00:00,MzBlQ7I3mfhoq87srrwl78MCuEU2,NaN,0.0,50.0
2,453829892.0,jasondavis,jasondavis,2024-11-24 23:06:47.564000+00:00,30.0,453829892.0,"Manila, Metro Manila, Philippines",signed,complete,not_hispanic,...,right,453829892.0,Tagalog (Filipino),['Asian / Asian-American'],female,2024-11-24 23:06:47.564000+00:00,MzBlQ7I3mfhoq87srrwl78MCuEU2,NaN,1.0,50.0
3,453829892.0,jasondavis,jasondavis,2024-11-24 23:06:47.564000+00:00,30.0,453829892.0,"Manila, Metro Manila, Philippines",signed,complete,not_hispanic,...,right,453829892.0,Tagalog (Filipino),['Asian / Asian-American'],female,2024-11-24 23:06:47.564000+00:00,MzBlQ7I3mfhoq87srrwl78MCuEU2,NaN,2.0,50.0
4,453829892.0,jasondavis,jasondavis,2024-11-24 23:06:47.564000+00:00,30.0,453829892.0,"Manila, Metro Manila, Philippines",signed,complete,not_hispanic,...,right,453829892.0,Tagalog (Filipino),['Asian / Asian-American'],female,2024-11-24 23:06:47.564000+00:00,MzBlQ7I3mfhoq87srrwl78MCuEU2,NaN,3.0,48.0
5,453829892.0,jasondavis,jasondavis,2024-11-24 23:06:47.564000+00:00,30.0,453829892.0,"Manila, Metro Manila, Philippines",signed,complete,not_hispanic,...,right,453829892.0,Tagalog (Filipino),['Asian / Asian-American'],female,2024-11-24 23:06:47.564000+00:00,MzBlQ7I3mfhoq87srrwl78MCuEU2,NaN,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175588,593527369.0,jasondavis,jasondavis,2024-11-19 03:29:41.867000+00:00,38.0,593527369.0,"Payson, Utah, USA",signed,complete,not_hispanic,...,right,593527369.0,English,['White / Caucasian'],female,2024-11-19 03:29:41.867000+00:00,z5mBRgqrEkda0UCfqIkXii3r3sw2,NaN,929.0,0.0
175589,593527369.0,jasondavis,jasondavis,2024-11-19 03:29:41.867000+00:00,38.0,593527369.0,"Payson, Utah, USA",signed,complete,not_hispanic,...,right,593527369.0,English,['White / Caucasian'],female,2024-11-19 03:29:41.867000+00:00,z5mBRgqrEkda0UCfqIkXii3r3sw2,NaN,930.0,0.0
175590,593527369.0,jasondavis,jasondavis,2024-11-19 03:29:41.867000+00:00,38.0,593527369.0,"Payson, Utah, USA",signed,complete,not_hispanic,...,right,593527369.0,English,['White / Caucasian'],female,2024-11-19 03:29:41.867000+00:00,z5mBRgqrEkda0UCfqIkXii3r3sw2,NaN,931.0,0.0
175591,593527369.0,jasondavis,jasondavis,2024-11-19 03:29:41.867000+00:00,38.0,593527369.0,"Payson, Utah, USA",signed,complete,not_hispanic,...,right,593527369.0,English,['White / Caucasian'],female,2024-11-19 03:29:41.867000+00:00,z5mBRgqrEkda0UCfqIkXii3r3sw2,NaN,932.0,0.0
